# LLM Zoomcamp - Homework 01
Emanuel Tomé\
June 2024

## Q1. Running Elastic

In [ ]:
# Answer: 42f05b9372a9a4a470db3b52817899b99a76ee73

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [2]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/home/erstome/LLM-zoomcamp-2024/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:06<00:00, 153.01it/s]


In [ ]:
# answer: index

## Q3. Searching

In [15]:
query = "How do I execute a command in a running docker container?"



In [10]:
# client = OpenAI()
from groq import Groq

from dotenv import load_dotenv

load_dotenv()
client = Groq()

In [47]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],#, "section"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    response_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        response_docs.append(hit)
    
    return result_docs, response_docs

In [48]:
result, response = elastic_search(query)

In [49]:
[dic['_score'] for dic in response]

[84.050095, 75.54128, 72.08518, 51.04628, 49.938507]

In [ ]:
# answer: 84.05

## Q4. Filtering

In [68]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],#, "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    response_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        response_docs.append(hit)
    
    return result_docs, response_docs

In [74]:
search_results, response = elastic_search(query)

In [75]:
search_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [76]:
# answer:
search_results[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [77]:
search_results[0]

{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I debug a docker container?',
 'course': 'machine-learning-zoomcamp'}

In [114]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [115]:
prompt = build_prompt(query, search_results)

In [116]:
len(prompt)

1463

## Q6. Tokens

In [118]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [120]:
encoded_prompt = encoding.encode(prompt)

In [121]:
len(encoded_prompt)

323

## Bonus: generating the answer

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        # model='gpt-4o',
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results, _ = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
response = rag(query)

In [ ]:
print(response)

A QUESTION from the FAQ database!

Given the CONTEXT from Module 1: Docker and Terraform, specifically the question regarding running pgcli in a Docker container, I'll answer your question:

To execute a command in a running Docker container, you can use the `docker exec` command. In your case, you can use:

`$ docker exec -it 175dd47cda07 pgcli -h pg-database -U root -p 5432 -d ny_taxi`

Replace `175dd47cda07` with the actual container ID or name.

Note that this command will execute the specified command (`pgcli`) inside the running container. The `-it` options allow you to interact with the container's standard input and output.
